In [78]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import load_cifar
import time

In [79]:
tf.reset_default_graph()

<h1>Define Hyper-perparmeter</h1>

In [80]:
#learning rate
#lr = 0.005
lr = 0.001
#number of traning steps
#num_steps =1750
#number of batch_size
#batch_size = 32


#network parameters
n_hidden_1 = 3000
n_hidden_2 = 3000
#n_hidden_3 = 1200
num_input = 3072
num_classes = 10

<h1>Define Placeholder</h1>

In [81]:

tf.reset_default_graph()

#tf graph input
X = tf.placeholder(tf.float32,[None,num_input],name='X')
Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')
#drop out rate
rate = tf.placeholder(tf.float32)

#Layers weight & bias
weights = {
    'W1': tf.Variable(tf.random_normal([num_input, n_hidden_1]),name='W1'),
    'W2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]),name='W2'),
    #'W3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3]),name='W3'),
    'Wout': tf.Variable(tf.random_normal([n_hidden_2, num_classes]),name='Wout')
}

biases = {
    'b1': tf.Variable(tf.zeros(shape=[n_hidden_1]),name='b1'),
    'b2': tf.Variable(tf.zeros(shape=[n_hidden_2]),name='b2'),
    #'b3': tf.Variable(tf.zeros(shape=[n_hidden_3]),name='b3'),
    'bout': tf.Variable(tf.zeros(shape=[num_classes]),name='bout')
}



<h1>Define Neural Network Architecture</h1>

In [82]:
#define a neural net model
def neural_net(x):
    
    layer_1_out = tf.add(tf.matmul(x,weights['W1']),biases['b1'])
    layer_1_out = tf.nn.relu(layer_1_out)
    drop_out = tf.layers.dropout(layer_1_out, rate)
    layer_2_out = tf.add(tf.matmul(drop_out, weights['W2']), biases['b2'])
    layer_2_out = tf.nn.relu(layer_2_out)
    drop_out = tf.layers.dropout(layer_2_out, rate)
    out = tf.add(tf.matmul(drop_out,weights['Wout']),biases['bout'])
    return out

<h1>Define cost andoptimization</h1>

In [83]:
#predicted labels
logits = neural_net(X)

#define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='loss')
#define optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

#compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))

#compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

#Initialize the variables
init = tf.global_variables_initializer()

Instructions for updating:
Use keras.layers.dropout instead.


<h1>Training and testing</h1>
<h2>1.Print out validation accuracy after each training poch</h2>
<h2>2.Print out training time you spend on each epoch</h2>
<h2>3.Print out testing accuracy in the end</h2>

In [84]:
valid_dict = pickle.load(open('processed_validate','rb'))
train_dict1 = pickle.load(open('processed_train1','rb'))
train_dict2 = pickle.load(open('processed_train2','rb'))
train_dict3 = pickle.load(open('processed_train3','rb'))
train_dict4 = pickle.load(open('processed_train4','rb'))
train_dict5 = pickle.load(open('processed_train5','rb'))
test_dict  = pickle.load(open('processed_test','rb'))

valid_features = valid_dict['normal feature']
valid_labels   = valid_dict['one hot labels']

train_features1 = train_dict1['normal feature']
train_labels1   = train_dict1['one hot labels']

train_features2 = train_dict1['normal feature']
train_labels2   = train_dict1['one hot labels']

train_features3 = train_dict1['normal feature']
train_labels3   = train_dict1['one hot labels']

train_features4 = train_dict1['normal feature']
train_labels4   = train_dict1['one hot labels']

train_features5 = train_dict1['normal feature']
train_labels5   = train_dict1['one hot labels']

test_features = test_dict['normal feature']
test_labels   = test_dict['one hot labels']

#Put all the training data together:
totalX = np.concatenate((train_features1,train_features2,train_features3,train_features4,train_features5))
totalY = np.concatenate((train_labels1,train_labels2,train_labels3,train_labels4,train_labels5))


In [87]:
with tf.Session() as sess:
    sess.run(init)
    
    num_steps = 50
    batch_size = 1000
    #train with the first training batch
    #for epoch in range(10)
    for epoch in range(100):
        start_time = time.time()
        for i in range(num_steps):
            #fetch batch
            #The input values have already been normalized
            batch_x_train = totalX[i*batch_size:(i+1)*batch_size,:]
            batch_y_train = totalY[i*batch_size:(i+1)*batch_size,:]

            #batch_x = train_features1[0:1028,:]
            #print(batch_size)
            #print(np.shape(batch_x_train))
            #run optimization
            sess.run(train_op, feed_dict={X:batch_x_train, Y:batch_y_train, rate:0.7})
            #if i % 100 ==0:
            acc = sess.run(accuracy,feed_dict={X:batch_x_train, Y:batch_y_train, rate:0.7})
            #print("In training batch: step "+str(i)+", Accuracy= {:.3f}".format(acc))
                #print(mnist.train.shape())
            if i % 10 ==0:
                #no drop out in validation
                print("Validating Accuracy:", sess.run(accuracy, feed_dict={X:valid_features, Y:valid_labels}))
                #print(sess.run(weights['W1'], feed_dict={X:valid_features, Y:valid_labels}))
        
        print("epoch {} is finished".format(epoch))
        elapsed_time = time.time() - start_time
        print("time used in the epoch is {}".format(elapsed_time))
        if epoch % 10 == 0:
            #no drop out in testing
            print("Testing Accuracy is :", sess.run(accuracy, feed_dict={X:test_features, Y:test_labels} ))
    print("Final testing accuracy is :", sess.run(accuracy, feed_dict={X:test_features, Y:test_labels} ))

Validating Accuracy: 0.1008
Validating Accuracy: 0.175
Validating Accuracy: 0.2466
Validating Accuracy: 0.259
Validating Accuracy: 0.2614
epoch 0 is finished
time used in the epoch is 21.81554079055786
Testing Accuracy is : 0.2541
Validating Accuracy: 0.2938
Validating Accuracy: 0.296
Validating Accuracy: 0.3152
Validating Accuracy: 0.3268
Validating Accuracy: 0.3268
epoch 1 is finished
time used in the epoch is 21.468738079071045
Validating Accuracy: 0.3526
Validating Accuracy: 0.3474
Validating Accuracy: 0.3532
Validating Accuracy: 0.3768
Validating Accuracy: 0.3722
epoch 2 is finished
time used in the epoch is 21.745580434799194
Validating Accuracy: 0.3838
Validating Accuracy: 0.3816
Validating Accuracy: 0.3926
Validating Accuracy: 0.393
Validating Accuracy: 0.4154
epoch 3 is finished
time used in the epoch is 21.54069757461548
Validating Accuracy: 0.4254
Validating Accuracy: 0.4228
Validating Accuracy: 0.3884
Validating Accuracy: 0.4332
Validating Accuracy: 0.4484
epoch 4 is finish